In [77]:
from models_to_prune import *
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import numpy as np

import torchvision.transforms as tf
import torchvision.datasets as ds
import torch.utils.data as data

import os
import time
import torch

# setup dataset
cifar10_trans = tf.Compose([tf.ToTensor(),
                            tf.Normalize((0.5,0.5,0.5),
                                         (0.5,0.5,0.5))])

train = ds.CIFAR10(root = os.getcwd(),
                   train = True,
                   download = True,
                   transform = cifar10_trans)
train_loader = data.DataLoader(train,
                               batch_size = 32,
                               shuffle = True,
                               num_workers = 0)

test = ds.CIFAR10(root = os.getcwd(),
                  train = False,
                  download = True,
                  transform = cifar10_trans)
test_loader = data.DataLoader(test,
                              batch_size = 64, # testing use less 
                                               # memory, can afford 
                                               # larger batch_size
                              shuffle = False,
                              num_workers = 0)


Files already downloaded and verified
Files already downloaded and verified


In [78]:
# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Train
model = BasicCNN()
model.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(0.001))

num_epochs = 50
best_val_acc = -1

print('Training CNN model')

for epoch in range(num_epochs):
    n_correct, n_total = 0, 0
    start_time = time.time()
    for idx, train_data in enumerate(train_loader):
        model.train()
        inputs, labels = train_data[0].to(device), train_data[1].to(device)

        optimizer.zero_grad()

        # forward
        preds = model(inputs) # forward pass
        loss = loss_func(preds,labels) # compute loss

        # backward
        loss.backward()  # compute grads
        optimizer.step() # update params w/ Adam update rule        

        # include batch accuracy to epoch training accuracy calculation
        _, prediction = torch.max(preds, dim=1) # idx w/ max val is
                                                # most confident class
        n_correct += (prediction == labels).sum().item()
        n_total += prediction.size(0)
        train_acc = n_correct/n_total

        # periodically evaluate performance on validation set 
        if idx % 500 == 0:            
            model.eval();           # switch model to evaluation mode
            correct, total = 0, 0   

            # calculate accuracy on validation set
            with torch.no_grad():
                for val_inputs, val_labels in test_loader:
                    val_inputs = Variable(val_inputs).cuda()
                    answer = model(val_inputs)
                    val_loss = loss_func(answer, val_labels)
                    _, val_preds = torch.max(answer, dim=1)
                    total += val_labels.size(0)
                    correct += (val_preds == val_labels).sum().item()
                val_acc = correct/total

            # print progress
            elapsed_time = time.time() - start_time
            str_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
            print('Epoch [{}/{}] Step [{}/{}] | Loss: {:.4f} Acc: {:.4f} Val_Loss: {:.4f} Val_Acc: {:.4f} Time: {}'
            .format(epoch+1, num_epochs, idx+1, len(train_loader), 
                    loss.item(), train_acc, 
                    val_loss.item(), val_acc,
                    str_time))

            # update best valiation set accuracy
            if val_acc >= best_val_acc:
                # found a model with better validation set accuracy
                snapshot_prefix = os.path.join(os.getcwd(), 'best_snapshot')
                snapshot_path = snapshot_prefix + '.pt'# '_val_acc_{}_val_loss_{}__epoch-iter_{}-{}_model.pt'.format(val_acc[-1], val_loss.item(), epoch, idx)

                # save model, delete previous 'best_snapshot' files
                torch.save(model, snapshot_path)
                best_val_acc = val_acc
                print("Model saved @ val acc =", best_val_acc)
            
print('Training Done')

Training CNN model
Epoch [1/50] Step [1/1563] | Loss: 2.3008 Acc: 0.1562 Val_Loss: 1.7109 Val_Acc: 0.1186 Time: 00:00:01
model saved @ val acc = 0.1186


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BasicCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6

Epoch [1/50] Step [501/1563] | Loss: 2.2242 Acc: 0.2987 Val_Loss: 1.7109 Val_Acc: 0.3267 Time: 00:00:07
model saved @ val acc = 0.3267
Epoch [1/50] Step [1001/1563] | Loss: 2.0114 Acc: 0.3125 Val_Loss: 1.7109 Val_Acc: 0.3468 Time: 00:00:14
model saved @ val acc = 0.3468
Epoch [1/50] Step [1501/1563] | Loss: 1.9725 Acc: 0.3251 Val_Loss: 1.7109 Val_Acc: 0.3697 Time: 00:00:20
model saved @ val acc = 0.3697
Epoch [2/50] Step [1/1563] | Loss: 2.0899 Acc: 0.3750 Val_Loss: 1.7109 Val_Acc: 0.3727 Time: 00:00:01
model saved @ val acc = 0.3727
Epoch [2/50] Step [501/1563] | Loss: 1.9917 Acc: 0.3750 Val_Loss: 1.7109 Val_Acc: 0.3806 Time: 00:00:07
model saved @ val acc = 0.3806
Epoch [2/50] Step [1001/1563] | Loss: 2.0550 Acc: 0.3760 Val_Loss: 1.7109 Val_Acc: 0.4127 Time: 00:00:14
model saved @ val acc = 0.4127
Epoch [2/50] Step [1501/1563] | Loss: 2.1417 Acc: 0.3795 Val_Loss: 1.7109 Val_Acc: 0.3903 Time: 00:00:20
Epoch [3/50] Step [1/1563] | Loss: 2.1785 Acc: 0.2812 Val_Loss: 1.7109 Val_Acc: 0.39

In [79]:
# Load Best Model
model = torch.load("best_snapshot.pt")
model.eval()

correct = 0
total = 0
# Test Model
with torch.no_grad():
    for val_inputs, val_labels in test_loader:
        val_inputs = Variable(val_inputs).cuda()
        answer = model(val_inputs)
        _, val_preds = torch.max(answer, dim=1)
        total += val_labels.size(0)
        correct += (val_preds == val_labels).sum().item()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))    

Accuracy of the network on the 10000 test images: 67 %


## All blocks below are just test codes

In [ ]:
cnn = BasicCNN()
for name, layer in cnn.named_modules():
    if 'conv' in name:
        filters = layer.weight.data.clone()
        print(name,':',filters.size())
        pooled_filter = torch.flatten(F.avg_pool2d(filters,
                                                   filters.size()[-1]))
        print("pooled :",pooled_filter.size())

In [ ]:
64*8

In [43]:
print(os.getcwd())

/content
